In [40]:
import pandas as pd

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)


In [3]:
data = pd.read_csv('/home/tyagi/Downloads/data-master/abcnews-date-text.csv', error_bad_lines=False);
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text


In [41]:
documents.shape

(1082168, 2)

In [42]:
documents[:5]

,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4


In [43]:
stemmer = SnowballStemmer('english')

In [44]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [45]:
doc_sample = documents[documents['index'] == 4310].values[0][0]

print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['rain', 'helps', 'dampen', 'bushfires']

 tokenized and lemmatized document: 
['rain', 'help', 'dampen', 'bushfir']


In [18]:
processed_docs = documents['headline_text'].map(preprocess)

In [19]:
processed_docs[:10]

0            [decid, communiti, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

To convert documents to vectors, we’ll use a document representation called bag-of-words. In this representation, each document is represented by one vector where each vector element represents a question-answer pair, in the style of:

    “How many times does the word system appear in the document?"

In [20]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [46]:
print(dictionary)

Dictionary(13917 unique tokens: ['broadcast', 'communiti', 'decid', 'licenc', 'awar']...)


In [21]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 communiti
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


In [24]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

The function doc2bow() simply counts the number of occurrences of each distinct word, converts the word to its integer word id and returns the result as a sparse vector

In [25]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(76, 1), (112, 1), (483, 1), (4009, 1)]

In [27]:
bow_doc_4310 = bow_corpus[4310]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0],dictionary[bow_doc_4310[i][0]], bow_doc_4310[i][1]))

Word 76 ("bushfir") appears 1 time.
Word 112 ("help") appears 1 time.
Word 483 ("rain") appears 1 time.
Word 4009 ("dampen") appears 1 time.


**tf-idf**

In [28]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)

In [29]:
corpus_tfidf = tfidf[bow_corpus]

In [30]:
from pprint import pprint

for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5902626795041239),
 (1, 0.3892065020004992),
 (2, 0.4955704490710528),
 (3, 0.5044979662918994)]


#### Running LDA using Bag of words

In [31]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [32]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))


Topic: 0 
Words: 0.030*"elect" + 0.027*"south" + 0.016*"say" + 0.015*"labor" + 0.014*"australia" + 0.014*"break" + 0.012*"meet" + 0.011*"minist" + 0.010*"royal" + 0.010*"hill"
Topic: 1 
Words: 0.022*"canberra" + 0.021*"death" + 0.021*"plan" + 0.017*"council" + 0.014*"concern" + 0.013*"north" + 0.013*"support" + 0.013*"road" + 0.013*"turnbul" + 0.012*"park"
Topic: 2 
Words: 0.027*"queensland" + 0.026*"year" + 0.017*"live" + 0.014*"farmer" + 0.012*"farm" + 0.011*"record" + 0.010*"green" + 0.010*"australia" + 0.009*"game" + 0.008*"launch"
Topic: 3 
Words: 0.030*"charg" + 0.027*"court" + 0.022*"murder" + 0.019*"crash" + 0.018*"face" + 0.016*"interview" + 0.015*"jail" + 0.015*"woman" + 0.015*"die" + 0.015*"alleg"
Topic: 4 
Words: 0.023*"market" + 0.016*"rise" + 0.016*"coast" + 0.016*"share" + 0.015*"price" + 0.015*"gold" + 0.014*"island" + 0.014*"high" + 0.014*"fall" + 0.013*"busi"
Topic: 5 
Words: 0.030*"govern" + 0.018*"rural" + 0.017*"school" + 0.016*"hour" + 0.015*"hospit" + 0.015*"heal

#### Running LDA using tf-idf

In [33]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)

In [34]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.022*"countri" + 0.020*"hour" + 0.019*"rural" + 0.016*"market" + 0.015*"trump" + 0.013*"news" + 0.011*"podcast" + 0.009*"share" + 0.008*"price" + 0.007*"turnbul"
Topic: 1 Word: 0.011*"kill" + 0.007*"peter" + 0.007*"hunter" + 0.006*"dead" + 0.006*"capit" + 0.006*"attack" + 0.006*"invest" + 0.005*"social" + 0.005*"valley" + 0.005*"harvest"
Topic: 2 Word: 0.010*"abbott" + 0.009*"hobart" + 0.009*"plead" + 0.009*"health" + 0.008*"mental" + 0.007*"legal" + 0.007*"action" + 0.006*"jam" + 0.006*"bail" + 0.006*"guilti"
Topic: 3 Word: 0.009*"australia" + 0.009*"world" + 0.008*"leagu" + 0.006*"rugbi" + 0.006*"dairi" + 0.006*"asylum" + 0.006*"august" + 0.005*"year" + 0.005*"seeker" + 0.005*"wall"
Topic: 4 Word: 0.012*"govern" + 0.007*"fund" + 0.007*"live" + 0.006*"plan" + 0.005*"council" + 0.005*"say" + 0.005*"coal" + 0.005*"budget" + 0.005*"wednesday" + 0.005*"energi"
Topic: 5 Word: 0.014*"coast" + 0.011*"weather" + 0.009*"gold" + 0.008*"sport" + 0.008*"donald" + 0.007*"juli" + 0.

#### Performance evaluation

In [35]:
processed_docs[4310]

['rain', 'help', 'dampen', 'bushfir']

In [36]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.5296608209609985	 
Topic: 0.027*"queensland" + 0.026*"year" + 0.017*"live" + 0.014*"farmer" + 0.012*"farm" + 0.011*"record" + 0.010*"green" + 0.010*"australia" + 0.009*"game" + 0.008*"launch"

Score: 0.3102540671825409	 
Topic: 0.030*"govern" + 0.018*"rural" + 0.017*"school" + 0.016*"hour" + 0.015*"hospit" + 0.015*"health" + 0.014*"hous" + 0.014*"tasmanian" + 0.013*"fund" + 0.011*"feder"

Score: 0.020011356100440025	 
Topic: 0.046*"polic" + 0.029*"sydney" + 0.025*"adelaid" + 0.018*"miss" + 0.018*"brisban" + 0.014*"investig" + 0.014*"fight" + 0.014*"children" + 0.013*"deal" + 0.013*"victoria"

Score: 0.02001091279089451	 
Topic: 0.023*"market" + 0.016*"rise" + 0.016*"coast" + 0.016*"share" + 0.015*"price" + 0.015*"gold" + 0.014*"island" + 0.014*"high" + 0.014*"fall" + 0.013*"busi"

Score: 0.020010557025671005	 
Topic: 0.030*"charg" + 0.027*"court" + 0.022*"murder" + 0.019*"crash" + 0.018*"face" + 0.016*"interview" + 0.015*"jail" + 0.015*"woman" + 0.015*"die" + 0.015*"alleg"

S

In [37]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.5745734572410583	 
Topic: 0.012*"drum" + 0.009*"john" + 0.009*"queensland" + 0.008*"octob" + 0.008*"christma" + 0.007*"tuesday" + 0.007*"novemb" + 0.007*"east" + 0.006*"decemb" + 0.006*"toni"

Score: 0.26531392335891724	 
Topic: 0.019*"charg" + 0.019*"polic" + 0.015*"murder" + 0.015*"court" + 0.011*"jail" + 0.010*"death" + 0.010*"woman" + 0.010*"alleg" + 0.009*"arrest" + 0.009*"shoot"

Score: 0.020022476091980934	 
Topic: 0.011*"kill" + 0.007*"peter" + 0.007*"hunter" + 0.006*"dead" + 0.006*"capit" + 0.006*"attack" + 0.006*"invest" + 0.005*"social" + 0.005*"valley" + 0.005*"harvest"

Score: 0.020015636458992958	 
Topic: 0.014*"coast" + 0.011*"weather" + 0.009*"gold" + 0.008*"sport" + 0.008*"donald" + 0.007*"juli" + 0.007*"michael" + 0.006*"care" + 0.005*"flood" + 0.005*"updat"

Score: 0.0200152900069952	 
Topic: 0.017*"interview" + 0.011*"crash" + 0.009*"road" + 0.008*"miss" + 0.008*"search" + 0.006*"truck" + 0.006*"mount" + 0.005*"plane" + 0.005*"river" + 0.005*"die"

Score: 

In [48]:
unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
  print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 5)))
  


Score: 0.1834942102432251	 
Topic: 0.046*"polic" + 0.029*"sydney" + 0.025*"adelaid" + 0.018*"miss" + 0.018*"brisban"

Score: 0.18347421288490295	 
Topic: 0.030*"elect" + 0.027*"south" + 0.016*"say" + 0.015*"labor" + 0.014*"australia"

Score: 0.1833483874797821	 
Topic: 0.027*"queensland" + 0.026*"year" + 0.017*"live" + 0.014*"farmer" + 0.012*"farm"

Score: 0.18332456052303314	 
Topic: 0.030*"charg" + 0.027*"court" + 0.022*"murder" + 0.019*"crash" + 0.018*"face"

Score: 0.18294648826122284	 
Topic: 0.025*"kill" + 0.019*"australian" + 0.019*"china" + 0.019*"women" + 0.015*"attack"

Score: 0.016682423651218414	 
Topic: 0.022*"canberra" + 0.021*"death" + 0.021*"plan" + 0.017*"council" + 0.014*"concern"

Score: 0.016682423651218414	 
Topic: 0.023*"market" + 0.016*"rise" + 0.016*"coast" + 0.016*"share" + 0.015*"price"

Score: 0.016682423651218414	 
Topic: 0.030*"govern" + 0.018*"rural" + 0.017*"school" + 0.016*"hour" + 0.015*"hospit"

Score: 0.016682423651218414	 
Topic: 0.027*"trump" + 0.0

In [49]:
unseen_document = 'Ideas are bulletproof Mr. Creeedy'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
  print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 5)))
  


Score: 0.5499468445777893	 
Topic: 0.030*"govern" + 0.018*"rural" + 0.017*"school" + 0.016*"hour" + 0.015*"hospit"

Score: 0.05000590905547142	 
Topic: 0.030*"elect" + 0.027*"south" + 0.016*"say" + 0.015*"labor" + 0.014*"australia"

Score: 0.05000590905547142	 
Topic: 0.022*"canberra" + 0.021*"death" + 0.021*"plan" + 0.017*"council" + 0.014*"concern"

Score: 0.05000590905547142	 
Topic: 0.027*"queensland" + 0.026*"year" + 0.017*"live" + 0.014*"farmer" + 0.012*"farm"

Score: 0.05000590905547142	 
Topic: 0.030*"charg" + 0.027*"court" + 0.022*"murder" + 0.019*"crash" + 0.018*"face"

Score: 0.05000590905547142	 
Topic: 0.023*"market" + 0.016*"rise" + 0.016*"coast" + 0.016*"share" + 0.015*"price"

Score: 0.05000590905547142	 
Topic: 0.027*"trump" + 0.024*"australia" + 0.022*"countri" + 0.021*"open" + 0.018*"test"

Score: 0.05000590905547142	 
Topic: 0.025*"kill" + 0.019*"australian" + 0.019*"china" + 0.019*"women" + 0.015*"attack"

Score: 0.05000590905547142	 
Topic: 0.024*"home" + 0.019*"